In [33]:
import add_packages
import os
from pprint import pprint
import pandas as pd
from operator import itemgetter

from my_configs import constants

from toolkit import sql
from toolkit.langchain import (
	document_loaders, text_splitters, text_embedding_models, stores,
	prompts, utils, output_parsers, agents, documents, runnables,
	histories, chains, models, tools
)

# Data

In [ ]:
my_sql_db = sql.SQLDatabase()

In [ ]:
schema_vinfast_cars = [
	"id SERIAL",
	"name VARCHAR(50) NOT NULL UNIQUE",
	"type VARCHAR(20) NOT NULL",
	"engine_type VARCHAR(20) NOT NULL",
	"webpage_link VARCHAR(200) NOT NULL",
	"specs TEXT NOT NULL",
	"brochure_link VARCHAR(200) NULL",
	"features TEXT NOT NULL",
	"interior_features TEXT NOT NULL",
	"exterior_features TEXT NOT NULL",
	"colors VARCHAR(200) NOT NULL",
	"price TEXT NOT NULL",
	"PRIMARY KEY (id)",
]

table_vinfast_cars = sql.SQLTable(
	name="vinfast_cars", 
	schema=schema_vinfast_cars,
	db=my_sql_db,
)

table_vinfast_cars.create()

In [ ]:
table_vinfast_cars.insert_from_excel(f"{add_packages.APP_PATH}/data/vinfast/cars.xlsx")

# Docs

## [Usecase - Build a Question/Answering system over SQL data](https://python.langchain.com/v0.2/docs/tutorials/sql_qa/)

In [ ]:
db = stores.SQLDatabase.from_uri(my_sql_db.get_uri())

In [ ]:
print(db.dialect)
print(db.get_usable_table_names())

### Chains


In [34]:
llm = models.chat_groq_mixtral

chain = chains.create_sql_query_chain(llm, db)

In [37]:
def query_processor(query: str):
	query = query.replace("\\", "")
	query = query.replace("SQLQuery: ", "")
	return query

In [ ]:
response = chain.invoke({"question": "How many cars are there?"})
response = query_processor(response)

In [35]:
db.run(response)

In [ ]:
chain.get_prompts()[0].pretty_print()

In [41]:
query_writer = chains.create_sql_query_chain(llm, db)
query_executor = tools.QuerySQLDataBaseTool(db=db)

prompt_answer = prompts.PromptTemplate.from_template("""\
Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer:\
""")

chain = (
	runnables.RunnablePassthrough.assign(query=query_writer).assign(
		result=itemgetter("query") 
					| runnables.RunnableLambda(query_processor) 
					| query_executor
	)
	| prompt_answer
	| llm
	| output_parsers.StrOutputParser()
)

In [42]:
chain.invoke({"question": "How many cars are there?"})

'There are 7 cars. The SQL result shows that the query returned a single row with a count of 7, indicating that there are 7 cars in the `vinfast_cars` table.'

### Agents


## Components

### Tools

#### [SQL Database](https://python.langchain.com/v0.2/docs/integrations/tools/sql_database/)


### Toolkits

#### [SQL Database](https://python.langchain.com/v0.2/docs/integrations/toolkits/sql_database/)


### 

## How-to

### [better prompt](https://python.langchain.com/v0.2/docs/how_to/sql_prompting/)

### [do query validation](https://python.langchain.com/v0.2/docs/how_to/sql_query_checking/)


### [deal with large databases](https://python.langchain.com/v0.2/docs/how_to/sql_large_db/)

### [do question answering over CSVs](https://python.langchain.com/v0.2/docs/how_to/sql_csv/)

## Cookbook

### [LLaMA2 chat with SQL](https://github.com/langchain-ai/langchain/blob/master/cookbook/LLaMA2_sql_chat.ipynb)

### [Databricks](https://github.com/langchain-ai/langchain/blob/master/cookbook/databricks_sql_db.ipynb)

### [Vector SQL Retriever with MyScale](https://github.com/langchain-ai/langchain/blob/master/cookbook/myscale_vector_sql.ipynb)

### [Incoporating semantic similarity in tabular databases](https://github.com/langchain-ai/langchain/blob/master/cookbook/retrieval_in_sql.ipynb)


### [SQL Database Chain](https://github.com/langchain-ai/langchain/blob/master/cookbook/sql_db_qa.mdx)


# Custom

# Todos

## Youtube

- [ ] https://alejandro-ao.com/chat-with-mysql-using-python-and-langchain/
- [ ] https://github.com/alejandro-ao/chat-with-mysql
- [ ] https://github.com/Coding-Crashkurse/Langchain-Dynamic-Routing
- [ ] https://python.langchain.com/v0.1/docs/integrations/toolkits/sql_database/
- [ ] https://blog.futuresmart.ai/mastering-natural-language-to-sql-with-langchain-nl2sql
- [ ] https://github.com/krishnaik06/Google-Gemini-Crash-Course/tree/main/sqlllm
- [ ] https://github.com/langchain-ai/langchain/tree/master/templates/sql-research-assistant
- [ ] https://www.rabbitmetrics.com/building-llm-knowledge-base-for-advanced-sql-chains/
- [ ] https://www.rabbitmetrics.com/chatting-with-ecommerce-data/
- [ ] https://github.com/Farzad-R/Advanced-QA-and-RAG-Series
- [ ] https://www.youtube.com/watch?v=XNeTgVEzILg (No code)
---
- [ ] https://www.youtube.com/watch?v=es-9MgxB-uc

## Docs
---
- [ ] Not Done
- [x] Done

### Tools